In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
from torch_geometric_temporal import METRLADatasetLoader
from other_model.other_model import make_default_model
import atd2022
torch.cuda.is_available()
torch.cuda.set_device(4)

In [2]:
atd_args = dotdict()

atd_args.name = 'atd'
atd_args.train_batch_size = 25
atd_args.predict_len = 4
atd_args.history_len = 7

atd_args.train_epochs= 10
atd_args.lr = 0.00009
atd_args.criterion = 'L1'
atd_args.scheduler = None

activation = 'relu'
pool_name = 'avg'
input_channel = 1
pixel_number = atd_args.history_len*5200
kernel_size = 6
knpp = 12
knpp2 = atd_args.predict_len

atd_args.layers=[
    ('agnostic', ((input_channel, knpp, kernel_size), 1, None, None, activation)),
    ('agnostic', ((knpp, knpp2, kernel_size), 1, None, None, activation)),
    ('linear', (knpp2*atd_args.history_len, atd_args.predict_len, (1,2), 1, (1, atd_args.predict_len))),
]

In [3]:
wt_args = dotdict()

wt_args.name = 'wiki_traffic'
wt_args.train_batch_size = 30
wt_args.predict_len = 1
wt_args.history_len = 5

wt_args.train_epochs= 10
wt_args.lr = 0.001
wt_args.criterion = 'L1'
wt_args.scheduler = None

activation = 'relu'
pool_name = 'avg'
input_channel = 1
pixel_number = wt_args.history_len*1400
kernel_size = 3
knpp = 5
knpp2 = 15

wt_args.layers=[
    ('specific', ((input_channel, knpp, kernel_size), 1, None, None, activation)),
    ('specific', ((knpp, knpp2, kernel_size), 1, None, None, activation)),
    ('linear', (knpp2*wt_args.history_len, wt_args.predict_len, (1,2), 1, (1, wt_args.predict_len))),
]

In [4]:
lat_args = dotdict()

lat_args.name = 'lat'
lat_args.train_batch_size = 30
lat_args.predict_len = 1
lat_args.history_len = 5

lat_args.train_epochs= 10
lat_args.lr = 0.001
lat_args.criterion = 'L1'
lat_args.scheduler = None

activation = 'relu'
pool_name = 'avg'
input_channel = 1
pixel_number = lat_args.history_len*207
kernel_size = 3
knpp = 5
knpp2 = 15

lat_args.layers=[
    ('specific', ((input_channel, knpp, kernel_size), 1, None, None, activation)),
    ('specific', ((knpp, knpp2, kernel_size), 1, None, None, activation)),
    ('linear', (knpp2*lat_args.history_len, lat_args.predict_len, (1,2), 1, (1, lat_args.predict_len))),
]

In [5]:
mnist_args = dotdict()

mnist_args.name = 'mnist'
mnist_args.train_batch_size = 120
mnist_args.predict_batch_size = 100


mnist_args.train_epochs = 30
mnist_args.lr = 0.001
mnist_args.criterion = 'CE'
mnist_args.optimizer = 'Adam'
mnist_args.scheduler = 'multistep3'
mnist_args.attack = {'fgsm':(0.1,), 'pgd':(0.1,0.1,20)}

activation = 'relu'
input_channel = 1
knpp = [30,60,120,180,240]

mnist_args.layers=[
    ('cnn2d', ((input_channel, knpp[0], (7,7), 1, 3, 1, 1), 1, None, None, activation, False)),
    ('atc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
    ('atc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
    ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, False)),
    ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, False, False)),
    ('adptavgpool', (1,1)),
    ('linear', (knpp[-1], 10, (1,2,3))),
    ('softmax', (1))
]

# knpp = [24,48,96,192]
# # knpp = [30,60,120,240]

# mnist_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
#     ('atc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('atc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, False, False)),
#     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3))),
#     ('softmax', (1))
# ]

In [6]:
cifar10_args = dotdict()

cifar10_args.name = 'cifar10'
cifar10_args.device = ['cuda:4', 'cuda:5']
cifar10_args.train_batch_size = 100
cifar10_args.predict_batch_size = 100

cifar10_args.train_epochs = 250
cifar10_args.lr = 0.01
cifar10_args.criterion = 'CE'
cifar10_args.optimizer = 'AdamW'
cifar10_args.scheduler = 'OneCycle'
cifar10_args.attack = {'fgsm':(0.005,), 'pgd':(0.005,0.1,20)}

activation = 'relu'
input_channel = 3
# knpp = [40,80,160,240,320]

# cifar10_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
#     ('atc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('atc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, False, False)),
#     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3))),
#     ('softmax', (1))
# ]

# resnet
# cifar10_args.layers=[
#     ('cnn2d', ((3, 16, (3, 3), 1, 1), 1, None, None, 'relu', False)), 
#     ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
#     ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
#     ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
#     ('cnn2d', ((16, 32, (3, 3), 1, 1), 2, 'first', (2, 2), 'relu')), 
#     ('cnn2d', ((32, 32, (3, 3), 1, 1), 2, None, None, 'relu')), 
#     ('cnn2d', ((32, 32, (3, 3), 1, 1), 2, None, None, 'relu')), 
#     ('cnn2d', ((32, 64, (3, 3), 1, 1), 2, 'first', (2, 2), 'relu')), 
#     ('cnn2d', ((64, 64, (3, 3), 1, 1), 2, None, None, 'relu')), 
#     ('cnn2d', ((64, 64, (3, 3), 1, 1), 2, None, None, 'relu')), 
#     ('adptavgpool', (1, 1)), 
#     ('linear', (64, 10, (1, 2, 3))), 
#     ('softmax', 1)
# ]

# abccnn
# knpp = [32,64,96,128,160,192,224]
# cifar10_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, 1), 2, None, None, activation, True)),
#     ('cnn2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, 1), 2, 'first', (2,2), activation, True)),
#     ('cnn2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, 1), 2, None, None, activation, True)),
#     ('cnn2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, 1), 2, 'first', (2,2), activation, True)),
#     ('cnn2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, 1), 2, None, None, activation, True)),
#     ('cnn2d', ((knpp[5], knpp[6], (3,3), 1, 1, 1, 1), 2, 'first', (2,2), activation, True)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, 1), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (2,2), 1, 0, 1, 1), 1, None, None, False, False)),
# #     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3))),
#     ('softmax', (1))
# ]


knpp = [48,96,144,240,336,432,528,624]

cifar10_args.layers=[
    ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, False)),
    ('atrc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, False)),
    ('atrc2d', ((knpp[5], knpp[6], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[6], knpp[7], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, False)),
    ('adptavgpool', (1,1)),
    ('linear', (knpp[-1], 10, (1,2,3)))
]

In [7]:
# torch.arange(1,10)*48

In [ ]:
args = cifar10_args
data = None
# data = atd2022.io.read_csv()
# data = pd.read_csv('/scratch/mfeng/data/ABC/Wiki_Traffic/filled_selected_train_1.csv', index_col=0, header=[0,1,2,3], parse_dates=True)
# data = pd.read_csv('/scratch/mfeng/data/ABC/LA_Traffic/LA_Traffic.csv', index_col=0)
# data = data.head(200)

driver = ABC_Driver(args, data, record_path=None, if_hash=False)
driver.train()

Use: ['cuda:4', 'cuda:5']
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
add record: 03/26/2023 09:48
epoch: 0, train_loss: 1.7715, test_metric: 0.5299, time: 640.1170034408569
epoch: 1, train_loss: 1.4424, test_metric: 0.6315, time: 637.0665020942688
epoch: 2, train_loss: 1.2745, test_metric: 0.6749, time: 636.9777297973633
epoch: 3, train_loss: 1.1583, test_metric: 0.7091, time: 637.0282580852509
epoch: 4, train_loss: 1.0692, test_metric: 0.7112, time: 636.8239636421204
epoch: 5, train_loss: 1.0016, test_metric: 0.7649, time: 637.0339353084564
epoch: 6, train_loss: 0.9575, test_metric: 0.7776, time: 637.1081812381744
epoch: 7, train_loss: 0.9159, test_metric: 0.7787, time: 637.2223207950592
epoch: 8, train_loss: 0.88, test_metric: 0.7857, time: 636.9696855545044
epoch: 9, train_loss: 0.8496, test_metric: 0.8027, time: 637.4048428535461
epoch: 10, train_loss: 0.828, test_metric: 0.8049, time: 637.2197160720825
epoch: 1

In [7]:
args = cifar10_args
data = None
# data = atd2022.io.read_csv()
# data = pd.read_csv('/scratch/mfeng/data/ABC/Wiki_Traffic/filled_selected_train_1.csv', index_col=0, header=[0,1,2,3], parse_dates=True)
# data = pd.read_csv('/scratch/mfeng/data/ABC/LA_Traffic/LA_Traffic.csv', index_col=0)
# data = data.head(200)

driver = ABC_Driver(args, data, record_path=None, if_hash=False)
driver.train()

Use: ['cuda:4', 'cuda:5']
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
add record: 03/21/2023 11:04


/scratch/mfeng/anaconda/envs/atd2022/lib/python3.9/site-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /opt/conda/conda-bld/pytorch_1670525539683/work/aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv2d(input, weight, bias, self.stride,


epoch: 0, train_loss: 2.0763, test_metric: 0.413, time: 289.0976550579071
epoch: 1, train_loss: 1.884, test_metric: 0.5155, time: 288.3746621608734
epoch: 2, train_loss: 1.7553, test_metric: 0.5697, time: 288.291672706604
epoch: 3, train_loss: 1.6458, test_metric: 0.6063, time: 288.28107476234436
epoch: 4, train_loss: 1.5531, test_metric: 0.6638, time: 288.22270011901855
epoch: 5, train_loss: 1.4642, test_metric: 0.6931, time: 288.1883018016815
epoch: 6, train_loss: 1.3969, test_metric: 0.703, time: 288.2314326763153
epoch: 7, train_loss: 1.315, test_metric: 0.735, time: 288.2426607608795
epoch: 8, train_loss: 1.2554, test_metric: 0.7478, time: 288.2106845378876
epoch: 9, train_loss: 1.2057, test_metric: 0.7549, time: 288.1534719467163
epoch: 10, train_loss: 1.1694, test_metric: 0.7699, time: 288.209885597229
epoch: 11, train_loss: 1.1257, test_metric: 0.7825, time: 288.2334530353546
epoch: 12, train_loss: 1.0954, test_metric: 0.7712, time: 288.1428027153015
epoch: 13, train_loss: 1.06

epoch: 108, train_loss: 0.3777, test_metric: 0.9412, time: 287.9883668422699
epoch: 109, train_loss: 0.3887, test_metric: 0.9437, time: 287.950807094574
epoch: 110, train_loss: 0.3817, test_metric: 0.943, time: 287.8674190044403
epoch: 111, train_loss: 0.379, test_metric: 0.9447, time: 287.7202353477478
epoch: 112, train_loss: 0.3799, test_metric: 0.946, time: 287.66538071632385
epoch: 113, train_loss: 0.3789, test_metric: 0.9443, time: 287.76473593711853
epoch: 114, train_loss: 0.3664, test_metric: 0.9436, time: 287.71680641174316
epoch: 115, train_loss: 0.3766, test_metric: 0.9458, time: 288.0416684150696
epoch: 116, train_loss: 0.3668, test_metric: 0.9424, time: 287.70735359191895
epoch: 117, train_loss: 0.3637, test_metric: 0.9414, time: 287.7299499511719
epoch: 118, train_loss: 0.3617, test_metric: 0.9441, time: 287.80285120010376
epoch: 119, train_loss: 0.3569, test_metric: 0.9471, time: 287.7107663154602
epoch: 120, train_loss: 0.3536, test_metric: 0.946, time: 287.7586705684662

epoch: 216, train_loss: 0.2067, test_metric: 0.9588, time: 287.6445314884186
epoch: 217, train_loss: 0.2021, test_metric: 0.9591, time: 287.7555732727051
epoch: 218, train_loss: 0.2063, test_metric: 0.961, time: 287.795289516449
epoch: 219, train_loss: 0.2037, test_metric: 0.9595, time: 287.84113788604736
epoch: 220, train_loss: 0.2048, test_metric: 0.9587, time: 287.84378814697266
epoch: 221, train_loss: 0.2069, test_metric: 0.9578, time: 287.67720079421997
epoch: 222, train_loss: 0.204, test_metric: 0.9584, time: 287.70468521118164
epoch: 223, train_loss: 0.2024, test_metric: 0.9594, time: 287.7460083961487
epoch: 224, train_loss: 0.2044, test_metric: 0.959, time: 287.6248080730438
epoch: 225, train_loss: 0.2018, test_metric: 0.9596, time: 287.7586979866028
epoch: 226, train_loss: 0.1963, test_metric: 0.9602, time: 287.6660256385803
epoch: 227, train_loss: 0.2032, test_metric: 0.9592, time: 287.7652473449707
epoch: 228, train_loss: 0.197, test_metric: 0.9597, time: 287.7005877494812


In [7]:
args = cifar10_args
data = None
# data = atd2022.io.read_csv()
# data = pd.read_csv('/scratch/mfeng/data/ABC/Wiki_Traffic/filled_selected_train_1.csv', index_col=0, header=[0,1,2,3], parse_dates=True)
# data = pd.read_csv('/scratch/mfeng/data/ABC/LA_Traffic/LA_Traffic.csv', index_col=0)
# data = data.head(200)

driver = ABC_Driver(args, data, record_path=None, if_hash=False)
driver.train()

Use: ['cuda:4', 'cuda:5']
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
add record: 03/16/2023 14:44
epoch: 0, train_loss: 1.9898, test_metric: 0.4628, time: 428.3412141799927
epoch: 1, train_loss: 1.3732, test_metric: 0.601, time: 430.7435441017151
epoch: 2, train_loss: 1.1166, test_metric: 0.6819, time: 430.48061871528625
epoch: 3, train_loss: 0.94, test_metric: 0.7446, time: 430.6922826766968
epoch: 4, train_loss: 0.8139, test_metric: 0.7514, time: 429.90587282180786
epoch: 5, train_loss: 0.7247, test_metric: 0.7844, time: 430.03293800354004
epoch: 6, train_loss: 0.6711, test_metric: 0.8005, time: 430.80587100982666
epoch: 7, train_loss: 0.6125, test_metric: 0.8136, time: 430.6304352283478
epoch: 8, train_loss: 0.5728, test_metric: 0.8156, time: 430.8235137462616
epoch: 9, train_loss: 0.5458, test_metric: 0.8321, time: 430.38902282714844
epoch: 10, train_loss: 0.5175, test_metric: 0.8316, time: 430.48436546325684
ep

epoch: 106, train_loss: 0.0205, test_metric: 0.923, time: 443.2584888935089
epoch: 107, train_loss: 0.0238, test_metric: 0.9302, time: 442.88967728614807
epoch: 108, train_loss: 0.0259, test_metric: 0.9195, time: 443.39852261543274
epoch: 109, train_loss: 0.0282, test_metric: 0.9257, time: 443.5046067237854
epoch: 110, train_loss: 0.0272, test_metric: 0.9271, time: 442.1726689338684
epoch: 111, train_loss: 0.0238, test_metric: 0.9259, time: 443.5874092578888
epoch: 112, train_loss: 0.0267, test_metric: 0.927, time: 442.9516682624817
epoch: 113, train_loss: 0.0306, test_metric: 0.9215, time: 442.9449760913849
epoch: 114, train_loss: 0.0365, test_metric: 0.924, time: 443.6515793800354
epoch: 115, train_loss: 0.0314, test_metric: 0.9169, time: 441.73201513290405
epoch: 116, train_loss: 0.0376, test_metric: 0.9248, time: 442.7366020679474
epoch: 117, train_loss: 0.0357, test_metric: 0.9233, time: 443.548152923584
epoch: 118, train_loss: 0.0311, test_metric: 0.9208, time: 442.9917645454407


epoch: 213, train_loss: 0.0008, test_metric: 0.9417, time: 443.6895661354065
epoch: 214, train_loss: 0.001, test_metric: 0.9414, time: 443.99684381484985
epoch: 215, train_loss: 0.0011, test_metric: 0.9428, time: 443.7667384147644
epoch: 216, train_loss: 0.0009, test_metric: 0.9412, time: 443.91667675971985
epoch: 217, train_loss: 0.0009, test_metric: 0.9411, time: 443.3837683200836
epoch: 218, train_loss: 0.0008, test_metric: 0.9428, time: 443.8919117450714
epoch: 219, train_loss: 0.0009, test_metric: 0.9414, time: 443.93101692199707
epoch: 220, train_loss: 0.0009, test_metric: 0.9411, time: 443.9579005241394
epoch: 221, train_loss: 0.0008, test_metric: 0.9421, time: 443.96986961364746
epoch: 222, train_loss: 0.0008, test_metric: 0.9422, time: 443.9425678253174
epoch: 223, train_loss: 0.0008, test_metric: 0.943, time: 443.950558423996
epoch: 224, train_loss: 0.0009, test_metric: 0.9409, time: 443.7249915599823
epoch: 225, train_loss: 0.0009, test_metric: 0.9425, time: 443.769679069519

In [8]:
# torch.save(driver.model.state_dict(), "save/CIFAR10_CNM_2023_03_22.pt")
# driver.model.load_state_dict(torch.load("save/CIFAR10_CNN_2023_03_07.pt"))

In [9]:
pytorch_total_params = sum(p.numel() for p in driver.model.parameters() if p.requires_grad)

In [10]:
pytorch_total_params

1649269

In [10]:
import foolbox as fb
import torch
import torchvision
from torchvision import datasets, transforms

preprocessing = dict(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010], axis=-3)
bounds = (0, 255)
driver.model.eval()
fmodel = fb.PyTorchModel(driver.model, bounds=bounds,preprocessing=preprocessing, device=torch.device('cuda'))
# attack = fb.attacks.LinfPGD(rel_stepsize=0.1, steps=20)
attack = fb.attacks.FGSM()

transform = transforms.Compose([transforms.ToTensor()])
testset = datasets.CIFAR10(root='../../data/ABC/CIFAR10', train=False, download=True, transform=transform)
predict = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

suc_all = 0
for i, (images, labels) in enumerate(predict):
    images = images.to(torch.device('cuda'))
    labels = labels.to(torch.device('cuda'))
    raw, clipped, is_adv = attack(fmodel, images, labels, epsilons=0.005)
#     raw, clipped, is_adv = attack(fmodel, images, labels, epsilons=0.031)
    suc = is_adv.sum()
    suc_all += suc
    print(f'{i}: {suc}')
attack_accuracy = 1 - suc_all/10000
print(attack_accuracy)

Files already downloaded and verified
0: 61
1: 62
2: 53
3: 67
4: 63
5: 62
6: 56
7: 67
8: 65
9: 61
10: 63
11: 60
12: 61
13: 57
14: 63
15: 62
16: 64
17: 60
18: 59
19: 60
20: 61
21: 57
22: 65
23: 62
24: 62
25: 68
26: 56
27: 59
28: 55
29: 62
30: 62
31: 53
32: 54
33: 67
34: 60
35: 55
36: 53
37: 64
38: 55
39: 62
40: 65
41: 62
42: 55
43: 63
44: 63
45: 69
46: 64
47: 57
48: 59
49: 63
50: 57
51: 65
52: 68
53: 53
54: 58
55: 67
56: 71
57: 56
58: 71
59: 60
60: 61
61: 61
62: 59
63: 51
64: 65
65: 66
66: 58
67: 65
68: 71
69: 59
70: 54
71: 63
72: 58
73: 60
74: 60
75: 56
76: 60
77: 52
78: 60
79: 61
80: 60
81: 62
82: 52
83: 66
84: 62
85: 62
86: 63
87: 65
88: 59
89: 62
90: 62
91: 58
92: 62
93: 60
94: 62
95: 55
96: 60
97: 63
98: 57
99: 57
tensor(0.3927, device='cuda:0')


In [ ]:

# cifar10_args.layers=[
#     ('agnostic', ((input_channel, knpp[0], 9), 1, None, None, activation, False)),
#     ('agnostic', ((knpp[0], knpp[0], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[0], knpp[1], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[1], knpp[2], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[2], knpp[3], 9), 2, 'first', (2,2), activation)),
#     ('agnostic', ((knpp[3], knpp[4], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[4], knpp[5], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[5], knpp[6], 9), 2, 'first', (2,2), activation)),
#     ('agnostic', ((knpp[6], knpp[7], 9), 2, None, None, activation)),
#     ('agnostic', ((knpp[7], knpp[8], 9), 2, None, None, activation)),
#     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3))),
#     ('softmax', (1))
# ]

# cifar10_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[0], knpp[0], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[0], knpp[1], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[1], knpp[2], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[2], knpp[3], (3,3), 1, 1), 2, 'first', (2,2), activation)),
#     ('cnn2d', ((knpp[3], knpp[4], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[4], knpp[5], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[5], knpp[6], (3,3), 1, 1), 2, 'first', (2,2), activation)),
#     ('cnn2d', ((knpp[6], knpp[7], (3,3), 1, 1), 2, None, None, activation)),
#     ('cnn2d', ((knpp[7], knpp[8], (3,3), 1, 1), 2, None, None, activation)),
#     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3))),
#     ('softmax', (1))
# ]

In [ ]:
[('cnn2d', ((3, 16, (3, 3), 1, 1), 1, None, None, 'relu', False)), 
 ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((16, 16, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((16, 32, (3, 3), 1, 1), 2, 'first', (2, 2), 'relu')), 
 ('cnn2d', ((32, 32, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((32, 32, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((32, 64, (3, 3), 1, 1), 2, 'first', (2, 2), 'relu')), 
 ('cnn2d', ((64, 64, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('cnn2d', ((64, 64, (3, 3), 1, 1), 2, None, None, 'relu')), 
 ('adptavgpool', (1, 1)), 
 ('linear', (64, 10, (1, 2, 3))), 
 ('softmax', 1)]